# Lab: Premier League matches analysis 

<img src="images/dataset-cover.jpg"></img>

## About Dataset
Context
Official football data organised and formatted in csv files ready for download is quite hard to come by. Stats providers are hesitant to release their data to anyone and everyone, even if it's for academic purposes. That was my exact dilemma which prompted me to scrape and extract it myself. Now that it's at your disposal, have fun with it.

## Content
The data was acquired from the Premier League website and is representative of seasons 2006/2007 to 2017/2018. Visit both sets to get a detailed description of what each entails.



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('premier-league-data.csv')
df

,home_team,away_team,home_goals,away_goals,result,season
0,Sheffield United,Liverpool,1,1,D,2006-2007
1,Arsenal,Aston Villa,1,1,D,2006-2007
2,Everton,Watford,2,1,H,?
3,Newcastle United,Wigan Athletic,2,1,H,2006-2007
4,Portsmouth,Blackburn Rovers,3,0,H,2006-2007
...,...,...,...,...,...,...
4555,Newcastle United,Chelsea,3,0,H,2017-2018
4556,Southampton,Manchester City,0,1,A,2017-2018
4557,Swansea City,Stoke City,1,2,A,2017-2018
4558,Tottenham Hotspur,Leicester City,5,4,H,2017-2018


> `result` could be `H: Home team won`, `A: Away team won`, `D: Draw`

---
## TASK 1

Can you find any match belonging to an invalid season?

If yes, go ahead and set "Unknown season" to the matches with invalid season.

**a) (correct)** 31 matches have an incorrect season value

b) 27 matches have an incorrect season value

c) 43 matches have an incorrect season value

d) There is not incorrect season values

---
### Hint

Check unique values for the `season` column and analyze if every value seems to be a valid season name.

---
### Solution

In [3]:
df['season'].value_counts()

2011-2012    380
2007-2008    380
2015-2016    380
2016-2017    380
2012-2013    380
2013-2014    380
2009-2010    380
2010-2011    380
2014-2015    380
2017-2018    380
2008-2009    380
2006-2007    349
?             31
Name: season, dtype: int64

In [4]:
df.loc[df["season"] == "?", 'season'] = "Unknown season"

In [5]:
df["season"].value_counts()

2011-2012         380
2007-2008         380
2015-2016         380
2016-2017         380
2012-2013         380
2013-2014         380
2009-2010         380
2010-2011         380
2014-2015         380
2017-2018         380
2008-2009         380
2006-2007         349
Unknown season     31
Name: season, dtype: int64

---
## TASK 2

Now it is time to check match scores. To do that we will analyze `home_goals` and `away_goals` columns to see if we find any match with wrong goals meassurement.

Did you find any match with invalid goals amount?

If yes, remove the entire row as we can't infer match results.

How many invalid values you found?

**a) (correct)** 34 invalid `home_goals` and 39 invalid `away_goals` values

b) 27 invalid `home_goals` and 43 invalid `away_goals` values

c) 41 invalid `home_goals` and 24 invalid `away_goals` values

d) 12 invalid `home_goals` and 19 invalid `away_goals` values

---
### Hint

Check the amount of `home_goals` and `away_goals` per match. A valid amount of goals is 0 or a positive amount.

---
### Solution

In [6]:
df['home_goals'].value_counts()

 1    1436
 2    1119
 0    1050
 3     568
 4     232
 5      75
 6      30
-2      21
 7      10
-1       9
 8       5
-4       4
 9       1
Name: home_goals, dtype: int64

In [7]:
df.loc[df['home_goals'] < 0].shape[0]

34

In [8]:
df = df.loc[df['home_goals'] >= 0]

In [9]:
df['away_goals'].value_counts()

 0    1549
 1    1532
 2     854
 3     380
 4     127
 5      31
-2      28
 6      13
-1       6
-4       5
 7       1
Name: away_goals, dtype: int64

In [10]:
df.loc[df['away_goals'] < 0].shape[0]

39

In [11]:
df = df.loc[df['away_goals'] >= 0]

---
## TASK 3

Match result `result` could be `H: Home team won`, `A: Away team won`, `D: Draw`.

Check if all the matches have a correct result value.

If you find any incorrect result, update it with the correct value.

How many incorrect match result values you found?


**a) (correct)** 43 incorrect match results

b) 38 incorrect match results

c) 47 incorrect match results

d) 32 incorrect match results

---
### Hint

You can check `home_goals` and `away_goals` to define the match `result`.

Use the `apply()` pandas method to iterate over every match and update the `result` column.

---
### Solution

In [12]:
df['result'].value_counts()

H    2055
A    1259
D    1130
?      43
Name: result, dtype: int64

In [13]:
def set_result(match):
    if match['home_goals'] > match['away_goals']:
        match['result'] = 'H'
    elif match['home_goals'] < match['away_goals']:
        match['result'] = 'A'
    else:
        match['result'] = 'D'
    
    return match
    
df.apply(lambda match: set_result(match), axis=1)['result'].value_counts()

H    2075
A    1269
D    1143
Name: result, dtype: int64

---
## TASK 4

What's the average amount of goals per match?

**a) (correct)** 2.68 goals per match

b) 2.34 goals per match

c) 1.89 goals per match

d) 3.13 goals per match

---
### Hint

---
### Solution

In [14]:
round(df.mean().sum(), 2)

2.68

---
## TASK 5

Calculate the average amount of goals per season.

Which is the season with the lowest average of goals?

**a) (correct)** Season 2006-2007

b) Season 2017-2018

c) Season 2009-2010

d) Season 2015-2016

---
### Hint

---
### Solution

In [15]:
df.groupby('season').mean().sum(axis=1).sort_values()

season
2006-2007         2.446377
2008-2009         2.470899
Unknown season    2.482759
2014-2015         2.558011
2007-2008         2.632000
2017-2018         2.678947
2015-2016         2.716981
2013-2014         2.749326
2011-2012         2.767380
2012-2013         2.794667
2009-2010         2.795148
2016-2017         2.796296
2010-2011         2.801587
dtype: float64

---
## TASK 6

What's the historical biggest goal difference?

**a) (correct)** 8 goals of difference

b) 10 goals of difference

c) 12 goals of difference

d) 14 goals of difference

---
### Hint

---
### Solution

In [16]:
abs(df['home_goals'] - df['away_goals']).sort_values(ascending=False)

1265    8
2458    8
1514    8
3116    8
1497    7
       ..
1110    0
1111    0
2365    0
4058    0
0       0
Length: 4487, dtype: int64

In [17]:
condition = list(df.reset_index()["index"].isin([1265, 2458, 1514, 3116]))

df.loc[condition]

,home_team,away_team,home_goals,away_goals,result,season
1265,Tottenham Hotspur,Wigan Athletic,9,1,H,2009-2010
1514,Chelsea,Wigan Athletic,8,0,H,2009-2010
2458,Chelsea,Aston Villa,8,0,H,2012-2013
3116,Southampton,Sunderland,8,0,H,2014-2015


---
## TASK 7

Which is the team with the most away wins? 

**a) (correct)** Chelsea

b) Manchester United

c) Arsenal

d) Liverpool

---
### Hint

---
### Solution

In [18]:
df.groupby('away_team').apply(lambda x: sum(x['result'] == 'A')).sort_values(ascending=False)

away_team
Chelsea                     118
Manchester United           116
Arsenal                     100
Manchester City              98
Liverpool                    94
Tottenham Hotspur            89
Everton                      64
Aston Villa                  49
West Ham United              41
Newcastle United             39
Sunderland                   34
Stoke City                   34
West Bromwich Albion         34
Southampton                  33
Swansea City                 30
Wigan Athletic               28
Crystal Palace               27
Blackburn Rovers             26
Bolton Wanderers             25
Fulham                       22
Leicester City               22
Portsmouth                   16
Watford                      15
AFC Bournemouth              13
Hull City                    12
Burnley                      12
Norwich City                 12
Reading                       9
Wolverhampton Wanderers       9
Birmingham City               8
Middlesbrough                 

---
## TASK 8

Wihch is the team with the most scored goals at home?

**a) (correct)** Manchester City

b) Manchester United

c) Arsenal

d) Chelsea

---
### Hint

---
### Solution

In [19]:
df[['home_team', 'home_goals']].groupby('home_team').sum().sort_values('home_goals', ascending=False)

,home_goals
home_team,
Manchester City,491
Chelsea,486
Manchester United,480
Arsenal,470
Liverpool,455
Tottenham Hotspur,413
Everton,389
West Ham United,281
Newcastle United,264
